In [5]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
import pandas as  pd
import requests
import numpy as np
from flipside import Flipside
import os
from dotenv import load_dotenv


In [4]:
# Config
columns = ['unique_trader', 'platform', 'blockchain','trading_vol', 'vol_all_platform', 'interactions_count', 'last_interact_date', 'last_date_all_platform', 'activity']

## Perpetual Trades

In [5]:
load_dotenv()

query_id = {
    "vertex": 3319571,
    "hmx": 3322162,
    "mux":3331840,
    "lvl":3329599,
    "synthetix":3331775,
    "gmx":3326970,
    "dydx":3350149,
    "aevo":3332234,
    "gns":3329779,
    'hyperliquid':3351533
}
dune_api = os.getenv('DUNE_API_2')
df_all = pd.DataFrame()
for platform,id in query_id.items():
    url = f"https://api.dune.com/api/v1/query/{str(id)}/results?api_key={dune_api}"
    response = requests.get(url)
    df = pd.DataFrame(response.json())
    data = df.iloc[-1,-1]
    df= pd.DataFrame(data)
    df= df[columns]
    # download
    df.to_excel(f"dataset/traders_{platform}.xlsx", index= False)
    print(f"donwload of {platform} data")
    # concatenate
    df_all = pd.concat([df_all,df], axis=0)
    

df_all.to_excel("dataset/concatenate.xlsx", index = False)
df_all

donwload of vertex data
donwload of hmx data
donwload of mux data
donwload of lvl data
donwload of synthetix data
donwload of gmx data
donwload of dydx data
donwload of aevo data
donwload of gns data
donwload of hyperliquid data


,unique_trader,platform,blockchain,trading_vol,vol_all_platform,interactions_count,last_interact_date,last_date_all_platform,activity
0,0xaf5e2af64e5926875fa5f3d5e6ac272fc602224e,Vertex,arbitrum,1.218857e+10,None,124255,2024-01-19,None,active
1,0x5494c260fd71799414d174aa802750efa2dca5dd,Vertex,arbitrum,1.123448e+10,None,117707,2024-01-19,None,active
2,0x351d1d0ed37f6854bfc36ba24caa529688990f6e,Vertex,arbitrum,3.344792e+09,None,122955,2024-01-19,None,active
3,0xd604cfa918a7fcf24fd5789483323cdcb37b7fcf,Vertex,arbitrum,5.960246e+08,None,28828,2024-01-19,None,active
4,0xc3239fbac3ecaff2165f5a27089fcf5d94c37fed,Vertex,arbitrum,5.326892e+08,None,22433,2024-01-19,None,active
...,...,...,...,...,...,...,...,...,...
1527,0xcfe122de90515685f806e79b29ff9351ce764098,Hyperliquid,optimism,1.000016e+04,None,1,2023-12-05,None,active
1528,0x9866f56b785d38e67eefed374821d1d7ae9ef18a,Hyperliquid,optimism,1.000013e+04,None,1,2023-10-13,None,active
1529,0x0681e5bee2f420bc5a1754f09e0dedc0c5b4be85,Hyperliquid,optimism,1.000006e+04,None,2,2023-12-05,None,active
1530,0xc18120b4b22c78ea55f8a7acbd7d7082d3c73a2b,Hyperliquid,optimism,1.000002e+04,None,2,2023-12-08,None,active


In [6]:
# Final Layout

pivot_df = df_all.pivot(index='unique_trader', columns='platform', values='trading_vol').fillna(0)
pivot_df['Cumulative Trading Volume'] = df_all.groupby('unique_trader')['trading_vol'].sum()
pivot_df['Count of Platforms Used'] = df_all.groupby('unique_trader')['platform'].nunique()
pivot_df['Number of Interactions'] = df_all.groupby('unique_trader')['interactions_count'].sum()
pivot_df['Date of Last Interaction'] = df_all.groupby('unique_trader')['last_interact_date'].max()
pivot_df.reset_index(inplace=True)


# Renaming the trading volume cols
platform_columns = [col for col in pivot_df.columns if col not in ['unique_trader', 'Cumulative Trading Volume', 'Count of Platforms Used', 'Number of Interactions', 'Date of Last Interaction']]
for col in platform_columns:
    pivot_df.rename(columns={col: f'Trading Volume {col}'}, inplace=True)

#pivot_df.to_excel("dataset/perpetual_dex.xlsx", index= False)  # or to_excel
pivot_df


platform,unique_trader,Trading Volume AAEVO,Trading Volume DYDX,Trading Volume GMX Trade,Trading Volume GNS,Trading Volume HMX,Trading Volume Hyperliquid,Trading Volume LVL Finance,Trading Volume MUX,Trading Volume Synthetix,Trading Volume Vertex,Cumulative Trading Volume,Count of Platforms Used,Number of Interactions,Date of Last Interaction
0,0x00000000000057667bfefe3158822e6fb4058a6d,0.0,0.0,58084.768813,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,5.808477e+04,1,3,2023-09-29
1,0x00000000000e8716b4e2d12400a01474cdfad760,0.0,0.0,0.000000,0.0000,0.0,0.0,7.527903e+06,0.0,0.0,0.000000e+00,7.527903e+06,1,1900,2023-09-27
2,0x000000000036277944858dfe6b9ae18c929e570c,0.0,0.0,0.000000,619294.1472,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,6.192941e+05,1,10,2024-01-09
3,0x00000000007cee88e01241c9db0a02b859db5b52,0.0,0.0,341570.146607,0.0000,0.0,0.0,4.952040e+05,0.0,0.0,0.000000e+00,8.367742e+05,2,1648,2023-12-25
4,0x00000000051cbce3fd04148cce2c0adc7c651829,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,2.548706e+04,2.548706e+04,1,6,2023-12-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46779,0xfffa9222fb4d2a2b6aaf99d55525281f27e03dd1,0.0,0.0,0.000000,26400.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.640000e+04,1,4,2024-01-12
46780,0xfffb0c1155a3268e6f2976a87083037adf0455e4,0.0,0.0,21115.553732,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.111555e+04,1,5,2023-08-16
46781,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d,0.0,0.0,29969.889880,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.996989e+04,1,15,2023-10-02
46782,0xfffb92fb89ae51c421018e42a5ca6e7336314438,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,1.024285e+06,1.024285e+06,1,68,2023-11-16


In [7]:
# Enrich perpetual traders table
# retrieve trader adresses vs type(contract or wallet)
url = "https://flipsidecrypto.xyz/api/v1/queries/1f6630a4-7d0c-4f70-a218-d603f5a094e9/data/latest"
response = requests.get(url)
data = response.json()
df_contract= pd.DataFrame(data)
display(df_contract)

df_perp = pd.merge(pivot_df, df_contract, how = 'left', left_on = 'unique_trader', right_on = 'UNIQUE_TRADER')


""


KeyError: 'UNIQUE_TRADER'

In [170]:
df_perp = df_perp.drop(columns = ['UNIQUE_TRADER'])
df_perp.to_excel("dataset/perpetual_dex.xlsx", index= False)
df_perp

,unique_trader,Trading Volume AAEVO,Trading Volume DYDX,Trading Volume GMX Trade,Trading Volume GNS,Trading Volume HMX,Trading Volume Hyperliquid,Trading Volume LVL Finance,Trading Volume MUX,Trading Volume Synthetix,Trading Volume Vertex,Cumulative Trading Volume,Count of Platforms Used,Number of Interactions,Date of Last Interaction,IS_ARB_CONTRACT,IS_OPT_CONTRACT,IS_BSC_CONTRACT
0,0x00000000000057667bfefe3158822e6fb4058a6d,0.0,0.0,58084.768813,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,5.808477e+04,1,3,2023-09-29,No,No,No
1,0x00000000000e8716b4e2d12400a01474cdfad760,0.0,0.0,0.000000,0.0000,0.0,0.0,7.527903e+06,0.0,0.0,0.000000e+00,7.527903e+06,1,1900,2023-09-27,No,No,Yes
2,0x000000000036277944858dfe6b9ae18c929e570c,0.0,0.0,0.000000,619294.1472,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,6.192941e+05,1,10,2024-01-09,No,No,No
3,0x00000000007cee88e01241c9db0a02b859db5b52,0.0,0.0,341570.146607,0.0000,0.0,0.0,4.952040e+05,0.0,0.0,0.000000e+00,8.367742e+05,2,1648,2023-12-25,No,No,No
4,0x00000000051cbce3fd04148cce2c0adc7c651829,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,2.548706e+04,2.548706e+04,1,6,2023-12-11,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46170,0xfffa9222fb4d2a2b6aaf99d55525281f27e03dd1,0.0,0.0,0.000000,26400.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.640000e+04,1,4,2024-01-12,No,No,No
46171,0xfffb0c1155a3268e6f2976a87083037adf0455e4,0.0,0.0,21115.553732,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.111555e+04,1,5,2023-08-16,No,No,No
46172,0xfffb43b6b6048adc32ded4fcedc429b17e7ea72d,0.0,0.0,29969.889880,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,0.000000e+00,2.996989e+04,1,15,2023-10-02,No,No,No
46173,0xfffb92fb89ae51c421018e42a5ca6e7336314438,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000e+00,0.0,0.0,1.024285e+06,1.024285e+06,1,68,2023-11-16,No,No,No


## NFT Community

In [8]:
#Extract data prepared from Flipside
url = "https://flipsidecrypto.xyz/api/v1/queries/e3cced65-01d9-48b9-b164-71a2675e3990/data/latest"
response = requests.get(url)
data = response.json()
df_nft= pd.DataFrame(data)
df_nft

,WALLET,NFT_NAME,COUNT,WALLET_BALANCE_USD
0,0xb7f7f6c52f2e2fdb1963eab30438024864c313f6,CRYPTOPUNKS,847,1.45
1,0x29469395eaf6f95920e59f858042f0e28d98a20b,PudgyPenguins,482,11526.62
2,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,CRYPTOPUNKS,420,20618.12
3,0xcb2a8334143176e6eb4863e7ab3ed5f3258ee94b,Merit Circle - Tactile NFT,247,15037.78
4,0x29469395eaf6f95920e59f858042f0e28d98a20b,Kanpai Pandas,236,11526.62
...,...,...,...,...
14595,0x10cc22fb156cc385946c83176885724b820e684c,CRYPTOPUNKS,1,13502.56
14596,0x8608ff4eb2f12f8ae0d18c3a82e6194950454a1a,PudgyPenguins,1,822.10
14597,0x85d14b5b709573c275b1ca9c1686cc120e9e715f,PudgyPenguins,1,10024.98
14598,0xce462c13f461deeb3ec2daadd5e7d5f90bb8f0b2,Cryptotester,1,3008.96


## Whales and OGs

###  TOP LProviders  on Uniswap & Curve AN (Ethereem and Arbitrum)

In [9]:
url = "https://flipsidecrypto.xyz/api/v1/queries/28a95d3b-2c12-42e7-97bf-aa2ac4c30e41/data/latest"
response = requests.get(url)
data = response.json()
df_dex= pd.DataFrame(data)
df_dex

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT,IS_ARB_CONTRACT
0,0x003fd5f6030b8c1a4d2503339b14466b9ba6c6cf,Uniswap & Curve,active,3.238479e+09,No,No
1,0x003db19049811190f330b20406fb1354fab27355,Uniswap & Curve,active,2.310849e+08,No,No
2,0x000000002fcecf3bc87acc58802cdb526c4f019d,Uniswap & Curve,active,2.281458e+06,No,No
3,0x0000000002ce79aacd54227d2163ff3791338975,Uniswap & Curve,active,3.396140e+06,No,No
4,0x000013de30d1b1d830dcb7d54660f4778d2d4af5,Uniswap & Curve,active,4.424423e+09,No,No
5,0x000bae21577c39cbb1e4efca262b6775ec1ac4c9,Uniswap & Curve,active,6.158243e+06,No,No
6,0x00000000ff0027b3f3d2b0412d4a4ced4e3366da,Uniswap & Curve,active,5.734032e+06,Yes,No
7,0x000f7f22bfc28d940d4b68e13213ab17cf107790,Uniswap & Curve,active,5.847530e+06,No,No
8,0x0004d2a2f9a823c1a585fde6514a17ff695e0001,Uniswap & Curve,active,6.479529e+06,No,No
9,0x0035fc5208ef989c28d47e552e92b0c507d2b318,Uniswap & Curve,active,7.145702e+06,No,No


### Staking Protocol such as Lido & Rocket Pool

In [10]:
url = "https://flipsidecrypto.xyz/api/v1/queries/b3239ae5-92c8-495d-99cc-9ac17b155dc8/data/latest"
response = requests.get(url)
data = response.json()
df_stake= pd.DataFrame(data)
df_stake

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x75a03ec24bf95f68a749d833a2efde50db7a6192,Lido & Rocketpool,active,3.526310e+07,Yes
1,0xa2e3a2ca689c00f08f34b62ccc73b1477ef1f658,Lido & Rocketpool,active,1.118856e+06,Yes
2,0x849d52316331967b6ff1198e5e32a0eb168d039d,Lido & Rocketpool,active,1.395823e+08,Yes
3,0xd2342aebf162b5852aadab82d6299fd5f57ad111,Lido & Rocketpool,active,3.532726e+06,Yes
4,0x5859e56aabaff9c31185da8c17d9fb52b3f091da,Lido & Rocketpool,active,1.064927e+06,No
...,...,...,...,...,...
1936,0xee8ad6769fe89ecb8fee0d981ad709e08e6d1c06,Lido & Rocketpool,active,3.231059e+07,No
1937,0xf91dc62ecb1d2cfc6ee72e1e7a27288c20bb5486,Lido & Rocketpool,active,2.957386e+06,No
1938,0xd3674a62851dcd14ad587b57615edf9d619df2c7,Lido & Rocketpool,active,2.571584e+06,No
1939,0xf54e7f23885082112f3f73e43429bd1c397c0963,Lido & Rocketpool,active,8.670041e+06,No


### MAkerDAO gov

In [11]:
url = "https://flipsidecrypto.xyz/api/v1/queries/8a8bd36e-e0a5-4170-a3c5-7fd16a50dbe6/data/latest"
response = requests.get(url)
data = response.json()
df_maker= pd.DataFrame(data)
df_maker

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD
0,0x918d368ea7bbe3af0f4ec9172802f8badae01284,Optimism Gov.,active,NaN
1,0xfe61acc408b63a5a03507a224398fa1fe8143f28,Optimism Gov.,active,NaN
2,0xc7da3b9cc5ef0435681876b035556477aff71318,Optimism Gov.,active,NaN
3,0xa4b28dd898a885cce88caa00261d9fe6ceed32bb,Optimism Gov.,active,NaN
4,0xfbd2923307a28803c2cb6cb53335596f3af7f00d,Optimism Gov.,active,NaN
...,...,...,...,...
298,0x5dc44a74b90b6221415f41f4fcfd652512139a03,Optimism Gov.,active,2.35
299,0x4a4bb53985ce40539b5cabdc6a9ae8fdf72d0a73,Optimism Gov.,active,2.35
300,0x4b8f1db532e427f2c5fe23f4920902c36ba1d98e,Optimism Gov.,active,0.86
301,0x3eb576f858b20085419fd3f09dee38e2ca372cd6,Optimism Gov.,active,0.37


### veCRV Holders

In [12]:
url = "https://flipsidecrypto.xyz/api/v1/queries/ba79099c-7652-4730-8e73-c077fd3e28bf/data/latest"
response = requests.get(url)
data = response.json()
df_crv= pd.DataFrame(data)
df_crv

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0xb2398330970c7cad95535d076ba3f64cc6afd43d,Curve Gov.,active,36777.161128,No
1,0x432dcbda06e8b296ca29705572d7cb6315ed8bed,Curve Gov.,active,10319.943538,No
2,0x7dc32a639ebb605d36f2c1076d5d74d69c254596,Curve Gov.,active,17425.734352,No
3,0x0991c61d1c55f2c77bb40be4ce223050fd0128ca,Curve Gov.,active,2451.250293,No
4,0x4f6fff5cd87eaf8cf09467366fe783e7ecb1317b,Curve Gov.,active,1466.316385,No
...,...,...,...,...,...
2777,0xc69750cc8e5f114d14aa8e5f92de994c612c58cd,Curve Gov.,active,2454.081402,No
2778,0xdbf5a8986e854fe07ffe4506e1d24d50fdf59974,Curve Gov.,active,4910.078619,No
2779,0x19ae3cf684087e2cb9cc2dd2b58c29f79f4e9d02,Curve Gov.,active,383523.884585,No
2780,0xfac2f11ba2577d5122dc1ec5301d35b16688251e,Curve Gov.,active,33941.914779,No


### Optimism Governance

In [13]:
url = "https://flipsidecrypto.xyz/api/v1/queries/4c6616d8-9808-40fa-9321-ccbe80bb20b9/data/latest"
response = requests.get(url)
data = response.json()
df_opt= pd.DataFrame(data)
df_opt

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD
0,0x50cac63559bad5c6a4a25155e3e6dde1e4b88893,Optimism Gov.,active,NaN
1,0x1929f1d365dc9d026a950a692d6c697b10235644,Optimism Gov.,active,NaN
2,0x4245a2a26c063a2abad002a895c18cfcdbad6d1c,Optimism Gov.,active,NaN
3,0xe4017ab0e6f3e98bf561edacec9687ade1aaf155,Optimism Gov.,active,NaN
4,0x47549016d259fdf677f2076bb61d871c6f07da83,Optimism Gov.,active,NaN
...,...,...,...,...
20781,0xa592c3ed459cd9a5283f66a73a577dac51da7608,Optimism Gov.,active,0.01
20782,0x71a89164464e2c76693c4675254288eb7006af3b,Optimism Gov.,active,0.01
20783,0x796cf37493529fba942e8c755f9bb4deb9cc882c,Optimism Gov.,active,0.01
20784,0x730a5b0dab84cb0baba70b0c8c7950986612757c,Optimism Gov.,active,0.01


### Governance (DEX/Stake/Chains)


In [19]:
url = "https://flipsidecrypto.xyz/api/v1/queries/631b19a0-46ac-4f2a-8a36-37be4cc41df2/data/latest"
response = requests.get(url)
data = response.json()
df_gov= pd.DataFrame(data)
df_gov['IS_ETH_CONTRACT'] = np.nan
df_gov

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0xadb10abb459b727a0236979c59af599e2e8d16e3,Aave Gov.,active,NaN,NaN
1,0x69c5c7b659368279c015bcc864741a85c5476f4c,Aave Gov.,active,NaN,NaN
2,0x4c50787793472f7dd4ba1b8802be75148437e48f,Aave Gov.,active,NaN,NaN
3,0xa80066e4244b917f1cfc5ba1831be06ac6138074,Aave Gov.,active,NaN,NaN
4,0xc5be9cc56b66e4ca0a07d05c3e309a557f678aab,Aave Gov.,active,NaN,NaN
...,...,...,...,...,...
60258,0xfc8295958b5a4317010fa9a9e54140377e95c87b,Aave Gov.,active,0.01,NaN
60259,0x57b8c52821931ada129110f5e70260f6ffce78e5,GMX Gov.,active,0.01,NaN
60260,0xa64969d5ecc9a609de90ebb8bc38b4e6656d0937,GMX Gov.,active,0.01,NaN
60261,0xbc30e87ddbbffb256b80cef2102d3d306856f147,GMX Gov.,active,0.01,NaN


### Gitcoin Grant Donors

In [17]:
dune_api = os.getenv("DUNE_API")
url = f"https://api.dune.com/api/v1/query/3351720/results?api_key={dune_api}"
response = requests.get(url)
df = pd.DataFrame(response.json())
data = df.iloc[-1,-1]
df_gitcoin= pd.DataFrame(data)
df_gitcoin = df_gitcoin[['wallet', 'platform', 'activity', 'amount_usd', 'is_eth_contract']]
df_gitcoin.columns = df_gitcoin.columns.str.upper()
df_gitcoin

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT
0,0x95305279414f309891342224e8afc192d4506cfa,Gitcoin Grants,active,0.658249,None
1,0x229549e5e046548271d1ac17ea9f4c27df15a3da,Gitcoin Grants,active,0.070032,None
2,0xf7b028006a3acb8a2293f4b895a9a6fa0bf071eb,Gitcoin Grants,active,0.000000,None
3,0xed9121f5bd8b953beea7ef27c212f0bdae12005f,Gitcoin Grants,active,0.069430,None
4,0x638c03201e2eb0d34e0f9ed14f4155192f786e9e,Gitcoin Grants,active,1.000792,None
...,...,...,...,...,...
1018,0x5b78f06123ee99dc8b21d4e1ef51afe08059d5ec,Gitcoin Grants,active,0.917005,None
1019,0x51e38d57d9231581760ce32a1b9947f906d8fa75,Gitcoin Grants,active,0.091639,None
1020,0x68ab70fe0a798bdb3583862e1dd9fb864fb9aa31,Gitcoin Grants,active,0.027787,None
1021,0xf87acaec6a9fe096e33592dbaaefb5c096ef5668,Gitcoin Grants,active,0.500903,None


## Final Layouts

In [20]:
df_og = pd.concat([df_stake, df_dex, df_maker, df_crv, df_opt, df_gov, df_gitcoin], axis = 0)
df_og.to_excel('dataset/og_whale_users.xlsx', index = False)
df_og

,WALLET,PLATFORM,ACTIVITY,AMOUNT_USD,IS_ETH_CONTRACT,IS_ARB_CONTRACT
0,0x75a03ec24bf95f68a749d833a2efde50db7a6192,Lido & Rocketpool,active,3.526310e+07,Yes,NaN
1,0xa2e3a2ca689c00f08f34b62ccc73b1477ef1f658,Lido & Rocketpool,active,1.118856e+06,Yes,NaN
2,0x849d52316331967b6ff1198e5e32a0eb168d039d,Lido & Rocketpool,active,1.395823e+08,Yes,NaN
3,0xd2342aebf162b5852aadab82d6299fd5f57ad111,Lido & Rocketpool,active,3.532726e+06,Yes,NaN
4,0x5859e56aabaff9c31185da8c17d9fb52b3f091da,Lido & Rocketpool,active,1.064927e+06,No,NaN
...,...,...,...,...,...,...
1018,0x5b78f06123ee99dc8b21d4e1ef51afe08059d5ec,Gitcoin Grants,active,9.170050e-01,None,NaN
1019,0x51e38d57d9231581760ce32a1b9947f906d8fa75,Gitcoin Grants,active,9.163950e-02,None,NaN
1020,0x68ab70fe0a798bdb3583862e1dd9fb864fb9aa31,Gitcoin Grants,active,2.778735e-02,None,NaN
1021,0xf87acaec6a9fe096e33592dbaaefb5c096ef5668,Gitcoin Grants,active,5.009028e-01,None,NaN


In [139]:
# NFT community final layout
perp_traders =  list(set(pivot_df.unique_trader.values))
og_whale_list =  list(set(df_og.WALLET.values))

df_nft['is_perp_trader'] = df_nft.WALLET.apply(lambda x: 'Yes' if x in perp_traders else 'No')
df_nft['is_og_whale'] = df_nft.WALLET.apply(lambda x: 'Yes' if x in og_whale_list else 'No')
df_nft.to_excel('dataset/nft_community.xlsx', index = False)
df_nft

,WALLET,NFT_NAME,COUNT,WALLET_BALANCE_USD,is_perp_trader,is_og_whale
0,0xb7f7f6c52f2e2fdb1963eab30438024864c313f6,CRYPTOPUNKS,829,1.19,No,No
1,0x29469395eaf6f95920e59f858042f0e28d98a20b,PudgyPenguins,440,10651.27,No,No
2,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,CRYPTOPUNKS,420,2458.25,No,No
3,0x29469395eaf6f95920e59f858042f0e28d98a20b,Kanpai Pandas,250,10651.27,No,No
4,0xcb2a8334143176e6eb4863e7ab3ed5f3258ee94b,Merit Circle - Tactile NFT,247,211834.42,No,No
...,...,...,...,...,...,...
14443,0x8b9f8a8a7c383f2d861cf05f35c28359cfbc97c3,lobsterdao,1,631.76,No,No
14444,0xff0613779201d09ec7f31f2fff5238d3998244e7,lobsterdao,1,774.98,No,No
14445,0xc4f370c6d3164a56971692362a9e488c0992a29d,CRYPTOPUNKS,1,1819.81,No,No
14446,0xec4f5fce4312936bae1da788d8ea595e13224df5,PudgyPenguins,1,3006.28,No,No


## Summary

In [123]:
df_con = pd.read_excel('dataset/concatenate.xlsx')
df_con

,unique_trader,platform,blockchain,trading_vol,vol_all_platform,interactions_count,last_interact_date,last_date_all_platform,activity
0,0xaf5e2af64e5926875fa5f3d5e6ac272fc602224e,Vertex,arbitrum,1.135845e+10,NaN,112298,2024-01-06,NaN,active
1,0x5494c260fd71799414d174aa802750efa2dca5dd,Vertex,arbitrum,9.769726e+09,NaN,105567,2024-01-06,NaN,active
2,0x351d1d0ed37f6854bfc36ba24caa529688990f6e,Vertex,arbitrum,2.724428e+09,NaN,110840,2024-01-06,NaN,active
3,0xd604cfa918a7fcf24fd5789483323cdcb37b7fcf,Vertex,arbitrum,4.762204e+08,NaN,24047,2024-01-06,NaN,active
4,0x519e19eac9ead404343c68224dd2a22c5c25f992,Vertex,arbitrum,4.017094e+08,NaN,4454,2024-01-03,NaN,active
...,...,...,...,...,...,...,...,...,...
49918,0xcfe122de90515685f806e79b29ff9351ce764098,Hyperliquid,optimism,1.000016e+04,NaN,1,2023-12-05,NaN,active
49919,0x9866f56b785d38e67eefed374821d1d7ae9ef18a,Hyperliquid,optimism,1.000013e+04,NaN,1,2023-10-13,NaN,active
49920,0x0681e5bee2f420bc5a1754f09e0dedc0c5b4be85,Hyperliquid,optimism,1.000006e+04,NaN,2,2023-12-05,NaN,active
49921,0xc18120b4b22c78ea55f8a7acbd7d7082d3c73a2b,Hyperliquid,optimism,1.000002e+04,NaN,2,2023-12-08,NaN,active


In [141]:
summary_perpetual = df_con.groupby(by = 'platform').agg({'unique_trader': 'count'})
display(summary_perpetual )
summary_nft = df_nft.groupby(by = 'NFT_NAME').agg({'WALLET': 'count'})
display(summary_nft )
summary_og = df_og.groupby(by = 'PLATFORM').agg({'WALLET': 'count'})
display(summary_og)

,unique_trader
platform,
AAEVO,1063
DYDX,2355
GMX Trade,22940
GNS,5356
HMX,1279
Hyperliquid,1533
LVL Finance,330
MUX,5009
Synthetix,4586


,WALLET
NFT_NAME,
CRYPTOPUNKS,3433
CryptoDickbutts S3,1819
Cryptotester,873
Kanpai Pandas,2107
Merit Circle - Tactile NFT,150
PudgyPenguins,4187
lobsterdao,1879


,WALLET
PLATFORM,
Aave Gov.,32680
Curve Gov.,2782
ENS Gov,1307
GMX Gov.,17514
Gitcoin Gov,5529
Gitcoin Grants,533
Lido & Rocketpool,1931
MakerDAO Gov.,316
Optimism,20786
